# Proyecto 1: Analizador de A/B Testing para una Web

En este proyecto, crearás un sistema en Python que analiza los resultados de un test A/B. Un test A/B es un experimento donde se muestran dos versiones de una página web (un grupo "A" y un grupo "B") a diferentes usuarios para ver cuál funciona mejor. Tu sistema leerá los datos de un archivo, calculará las estadísticas clave para cada grupo y declarará un ganador.

### Paso 1: Prepara tus Datos (Crea el archivo `test_ab_data.csv`)

Crea un archivo llamado `test_ab_data.csv` en la misma carpeta que tu notebook con datos que simulen el tiempo (en segundos) que los usuarios pasaron en cada versión de la página. Debe tener las columnas: `id_usuario`, `grupo`, `tiempo_en_pagina`.

### Paso 2: Define la Clase `GrupoExperimental`

Esta clase representará a un grupo del test (A o B) y será responsable de analizar sus propios datos.

* **Constructor `__init__`**: Debe recibir `nombre` y `dataset` (una lista de números).
* **Método `calcular_estadisticas()`**: Debe calcular la media, mediana, desviación estándar, mínimo y máximo del `dataset` y devolver un diccionario con los resultados.
* **Método `__str__()`**: Debe devolver un string bien formateado que resuma las estadísticas del grupo.

### Paso 3: Define la Clase `AnalizadorAB`

Esta es la clase principal que orquestará todo el análisis.

* **Constructor `__init__`**: Debe recibir `nombre_test`.
* **Método `cargar_datos_desde_csv(ruta_archivo)`**: Debe leer el archivo CSV, separar los datos para cada grupo y crear dos objetos de la clase `GrupoExperimental`.
* **Método `declarar_ganador()`**: Debe usar los objetos de grupo para calcular sus estadísticas, imprimirlas y comparar una métrica clave (la media) para determinar un ganador.

### Paso 4: Ejecuta el Análisis

Crea el código final que ponga todo en marcha:
1.  Crea una instancia de `AnalizadorAB`.
2.  Llama a `cargar_datos_desde_csv()` para cargar los datos del archivo.
3.  Llama a `declarar_ganador()` para ver los resultados.

---

# Solución

---

## Proyecto Final: Solución del Analizador de A/B Testing

**El Objetivo:** Crear un sistema en Python que lee los resultados de un test A/B desde un archivo CSV, calcula las estadísticas descriptivas para cada grupo (A y B) y declara un ganador basado en una métrica clave.

## 1. Preparación de Datos

Como primer paso, creamos nuestro archivo `test_ab_data.csv` para tener una fuente de datos con la que trabajar. Esto hace que nuestro notebook sea autocontenido y reproducible.

In [1]:
# Importamos la librería para manejar archivos CSV.
import csv

# Definimos los datos que contendrá nuestro archivo.
datos_csv = [
    ['id_usuario', 'grupo', 'tiempo_en_pagina'],
    ['user1', 'A', '120'],
    ['user2', 'B', '135'],
    ['user3', 'A', '110'],
    ['user4', 'B', '145'],
    ['user5', 'A', '95'],
    ['user6', 'B', '150'],
    ['user7', 'A', '115'],
    ['user8', 'B', '125'],
    ['user9', 'A', '130'],
    ['user10', 'B', '160']
]

# Usamos with open() para crear y escribir en el archivo de forma segura.
# 'w' es el modo de escritura (write) y newline='' es una buena práctica para archivos CSV.
with open('test_ab_data.csv', 'w', newline='') as file:
    # Creamos un objeto 'writer' para escribir en el archivo.
    writer = csv.writer(file)
    # El método .writerows() escribe todas las filas de nuestra lista de datos.
    writer.writerows(datos_csv)

# Imprimimos un mensaje para confirmar que el archivo se creó.
print("Archivo 'test_ab_data.csv' creado con éxito.")

Archivo 'test_ab_data.csv' creado con éxito.


## 2. Creación de los "Planos" (Clases)

Diseñamos nuestras clases. La clase `GrupoExperimental` se encargará de los cálculos para un solo grupo, y `AnalizadorAB` orquestará todo el proceso.

In [ ]:
# Importamos la librería de statistics para los cálculos.
import statistics
# Importamos copy para crear copias seguras de nuestros datos.
import copy

# --- PLANO 1: Para analizar un solo grupo ---
class GrupoExperimental:
    # El constructor. Se ejecuta al crear un objeto GrupoExperimental.
    def __init__(self, nombre, dataset):
        # Guarda el nombre del grupo (ej. 'A') como un atributo del objeto.
        self.nombre = nombre
        # Guarda la lista de datos (tiempos) como un atributo.
        self.dataset = dataset

    # Método que realiza todos los cálculos estadísticos para este grupo.
    def calcular_estadisticas(self):
        # Devuelve un diccionario con todos los resultados para un fácil acceso.
        return {
            'media': statistics.mean(self.dataset),      # El promedio de los valores.
            'mediana': statistics.median(self.dataset),    # El valor central de los datos ordenados.
            'std_dev': statistics.stdev(self.dataset),   # La desviación estándar (dispersión).
            'minimo': min(self.dataset),               # El valor más pequeño.
            'maximo': max(self.dataset)                # El valor más grande.
        }

    # Método especial que define cómo imprimir el objeto de forma legible.
    def __str__(self):
        # Llama a su propio método para obtener las estadísticas.
        stats = self.calcular_estadisticas()
        # Devuelve un string formateado con los resultados clave.
        return f"--- Resumen del {self.nombre} ---\nMedia: {stats['media']:.2f}\nMediana: {stats['mediana']}\nDesv. Estándar: {stats['std_dev']:.2f}"


# --- PLANO 2: Para orquestar todo el análisis ---
class AnalizadorAB:
    # Constructor que inicializa el análisis con un nombre.
    def __init__(self, nombre_test):
        # Guarda el nombre del experimento.
        self.nombre_test = nombre_test
        # Prepara los atributos para guardar los objetos de cada grupo, inicialmente vacíos.
        self.grupo_a = None
        self.grupo_b = None

    # Método para leer el archivo y organizar los datos.
    def cargar_datos_desde_csv(self, ruta_archivo):
        # Creamos listas temporales para separar los datos de cada grupo.
        datos_a = []
        datos_b = []

        # Abrimos el archivo CSV en modo lectura ('r').
        with open(ruta_archivo, mode='r', newline='') as file:
            # Usamos DictReader para leer cada fila como un diccionario.
            lector_csv = csv.DictReader(file)
            # Iteramos sobre cada fila del archivo CSV.
            for fila in lector_csv:
                # Usamos un condicional para dirigir cada dato a su lista correspondiente.
                if fila['grupo'] == 'A':
                    # Convertimos el tiempo a float y lo añadimos a la lista del grupo A.
                    datos_a.append(float(fila['tiempo_en_pagina']))
                elif fila['grupo'] == 'B':
                    # Convertimos el tiempo a float y lo añadimos a la lista del grupo B.
                    datos_b.append(float(fila['tiempo_en_pagina']))

        # Creamos los objetos, pasando los datos que acabamos de separar.
        self.grupo_a = GrupoExperimental("Grupo A", datos_a)
        self.grupo_b = GrupoExperimental("Grupo B", datos_b)
        # Imprimimos un mensaje de confirmación.
        print("Datos cargados y grupos experimentales creados.")

    # Método principal que ejecuta y presenta el análisis.
    def declarar_ganador(self):
        # Imprime un encabezado para el reporte.
        print(f"\n===== RESULTADOS DEL TEST: {self.nombre_test} =====")

        # Imprime el resumen del Grupo A (esto llama al método __str__ de self.grupo_a).
        print(self.grupo_a)
        # Imprime el resumen del Grupo B (esto llama al método __str__ de self.grupo_b).
        print(self.grupo_b)

        # Obtenemos los diccionarios de estadísticas para poder compararlos.
        stats_a = self.grupo_a.calcular_estadisticas()
        stats_b = self.grupo_b.calcular_estadisticas()

        # Imprime una línea divisoria para separar los resultados.
        print("\n--- Conclusión ---")
        # Compara la métrica clave (la media) para determinar el ganador.
        if stats_a['media'] > stats_b['media']:
            # Declara al Grupo A como ganador si su media es mayor.
            print("El ganador es el Grupo A, con un mayor tiempo promedio en la página.")
        elif stats_b['media'] > stats_a['media']:
            # Declara al Grupo B como ganador si su media es mayor.
            print("El ganador es el Grupo B, con un mayor tiempo promedio en la página.")
        else:
            # Declara un empate si las medias son idénticas.
            print("No hay un ganador claro, ambos grupos tuvieron el mismo tiempo promedio.")
        # Imprime un pie de página para el reporte.
        print("=" * 53)

## 3. Simulación: Poniendo Todo en Marcha

In [ ]:
# 1. Creamos una instancia del analizador, dándole un nombre a nuestro test.
test_landing_page = AnalizadorAB("Test de Tiempo en Página de Aterrizaje")

# 2. Llamamos al método para que cargue y procese los datos del archivo.
test_landing_page.cargar_datos_desde_csv('test_ab_data.csv')

# 3. Llamamos al método que realiza la comparación y muestra el reporte final.
test_landing_page.declarar_ganador()